# Installing the ComAp package
1. You need to have `python` 3.11 or newer installed
2. Download the latest release from the `ComAp-API` github repository
3. Get the `ComAp-Key`, `client_id` and `secret` and add them to the `.env.secret` file, together with yout WSV user name.
4. For the run the `test-units`, get a genset GUID and add it to the `.env.shared` file - the examples talking to a specific genset will use that.
5. Also, for `test-download`, run `test-files` first, pick a filename and also put it in the `.env.shared`

# Authentication - API secrets
Each WSV API call needs to have three parameters: `ComAp-Key`, authentication `Token` and your WebSupervisor `user name` (since there can be multiple users for one ComAp cloud identity).

# ComAp-Key
To get the ComAp-Key, login to https://portal.websupervisor.net/developer (using your WSV Pro login credentials). There are two keys available - Primary and Secondary - pick one of them - it does not matter which one.

*(The idea is, that you use one of them in production, and the second for testig. After the testing, you can Regenerate the key, to make sure it stays secret. And this will not impact the other key that is used in production.)*

You can generate a new `ComAp-Key` from the portal.

### Generating token
The `Token` is generated by calling ComAp Cloud Identity API `Authenticate`. This authentication needs three parameters - `client_id`, `secret` and again `ComAp-Key`.
The `client_id` and `secret` will be available on your customer portal in 2024. Right now, they are not there, so you can use the API documentation portal to generate them - as a workaround until then. Login to the API documentation portal, and use the `Try it` button to call the following APIs. You will need enter your login details and MFA:

1. [Create Application Registration](https://portal.websupervisor.net/docs/services/comap-cloud-identity/operations/application-create?). In the Authorization pick Implict. Complete the login, then Send the request - this will return the `client_id`.
2. [Create Application Secret](https://portal.websupervisor.net/docs/services/comap-cloud-identity/operations/application-secret-create?). In the body, fill-in the `client-id`, again, pick Implicit Authorizationm then send the request - this will return the `secret`

Save these two values in the `.env.secret` - there is no way to get them. If you need new ones, [Delete Application Registration](https://portal.websupervisor.net/docs/services/comap-cloud-identity/operations/application-delete?) and create new registration. These values are valid for 2 years.


### Storing the API secrets 
The API key should not be hard-coded in the script. There are number of ways for storing the keys securely, I have chosen to use the [`dotenv_values`](https://pypi.org/project/python-dotenv/) library to save them in the .env files (that should be kept secret) and read them at the begginning of each program:

In [6]:
from dotenv import dotenv_values

# Retrieve secrets and common values stored in the .env files
secrets = dotenv_values(".env.secret")
shared = dotenv_values(".env.shared")

# Authentication
This will create a bearer token.

In [7]:
from comap import api

identity = api.Identity(secrets["COMAP_KEY"])
token = identity.authenticate(secrets["CLIENT_ID"], secrets["SECRET"])

This will return a `dict` with the information about the token. The actual token is stored in `token["access_token"]`

# Calling WebSupervisor
All ComAp WebSupervisor APIs are methods of `wsv` class. To use them, you need to create an instance of this class, providing your WSV user name, the ComAp Key and the token from the previous step.

In [8]:
wsv = api.WSV(secrets["LOGIN_ID"],secrets["COMAP_KEY"], token["access_token"])

Then you can use individual functions.

## Units
To get the list of units (gensets) under your WSV account, call method `wsv.units`.

It returns a list of units, each with the following attributes: `name`, `unitGuid`, `url`.

In [9]:
for unit in wsv.units():
    print(unit["name"])

Hybrid Barcelona
IL-NT
ID FLX lite
Cogeneration unit ABB Prague
Westport 
ID-Lite
IA-NT STD test
IC-NT Mint 2
Cogeneration unit Hilton London
ID-EM
Harvester Goa
Habana hospital
AIO GAS
New marine
IL9
Controller Solo
Hybrid
test33
sifrovani IL3
Global
C07 - Hybrid Master
IG200_VAJL
HYBRID ComAp headquarter Prague


## Info
This is to get the information about an unit. For this method (and all the following ones, we need to specify the unit by its `unitGuid`. You can get this information from calling units, or from the WebSupervisor web unit detail url.

For this example, let's use the Genset ID that you stored in the `.env.shared` file.

In [10]:
unitGuid = shared["GENSET_ID"]
info = wsv.info(unitGuid)
values = [('Name', info['name']),
          ('Owner', info['ownerLoginId']),
          ('Timezone', info['timezone']),
          ('Position', f'{info["position"]["latitude"]},{info["position"]["longitude"]}')
]
for desc, value in values:
    print(f'{desc:10}{value}')

Name      Westport 
Owner     StaceyH5110
Timezone  America/New_York
Position  40.768301,-73.989698


## Values
WIth this you can read controller values. You can read all values (not recommended) or list of specific values. To specify the values to get, you need to include comma separated list of their `valueGuid`s. 

The list of available value depends on the type of controller. You can get the list of available `valueGuid`s by calling it without any `valueGuid`s.

The list of common `valueGuid`s is available from constant `VALUE_GUID`. Let's check it:

In [11]:
from comap.constants import VALUE_GUID
for value in VALUE_GUID:
    print(f'{value:18} {VALUE_GUID[value]}')

genset_name        7b2ae258-65a8-40dd-bb42-5455753679f9
serial_number      6253525D-CF01-4a15-A34B-6C232496E7B4
last_update        0C9117DA-495A-11DF-85EB-428556D89593
comm_state         F0219C1C-1860-4b4d-8E6E-3CEC96279D6F
unit_state         48EC10D1-7AF8-4ce9-B5B3-A5315993FEB3
unit_changed       52C8C105-49E6-477B-B43D-33FD46F54640
controller         B9FF3CE7-0A81-4D80-8029-99C24A0E764F
alarm_list         7EDE9575-FEB5-4540-9153-A3B9CF60DE9F
alarm_list_ext     5E36C1AA-A2B7-4264-96B5-BB975E48B547
mode               6a12aed6-3be0-44c4-9110-9ea33cfe3ccc
nominal_power      72D0295A-3E65-11DF-892C-D6A856D89593
actual_power       EE83E7E9-7453-4e64-A5CB-C8C093FB2A2F
engine_state       BB2D1ADE-740E-488d-853B-6BA970D52E27
breaker_state      B7E5B9B1-9F5E-45c1-82F4-3A336F1F97FA
Gcb_mode           b0f910f3-14e4-4bd1-8db1-739e26680b9c
Mcb_mode           5bbc94e2-2497-4d2b-a3e6-1c798d0b6430
import_kWh         4C214D52-3D75-11DF-8AA2-529056D89593
U_L1               4a2a2694-3049-4a8e-ba5f-cc6fe

Now let's read few values:

In [12]:
values = wsv.values(
    unitGuid,
    f'{VALUE_GUID["genset_name"]},'
    f'{VALUE_GUID["mode"]},'
    f'{VALUE_GUID["engine_state"]}'
)
for value in values:
    print(f'{value["name"]:<16} : {value["value"]}')

Gen-set name     : 2692 Westport
ControllerMode   : AUT
Engine state     : Loaded


## History
To get log of the values for given time period (do not confuse with the controller history)

In [13]:
history = wsv.history(
    unitGuid,
    '11/01/2023',
    '11/02/2023',
    VALUE_GUID["engine_state"]
)
for event in history[0]["history"]:
    print(f'{event["value"]:<8} valid to {event["validTo"]}')

Loaded   valid to 2023-11-05 04:00:00+00:00


## Files
List the files (reports and controller history) stored on the controller.

In [14]:
for file in wsv.files(unitGuid):
    print(f'{file["generated"]} {file["fileType"]} [{file["fileName"]}]')

2018-01-24 06:00:10+00:00 unitHistory [2018-01-24_01-00_Westport .csv]
2018-01-25 06:00:11+00:00 unitHistory [2018-01-25_01-00_Westport .csv]
2018-01-26 06:00:10+00:00 unitHistory [2018-01-26_01-00_Westport .csv]
2018-01-27 06:00:11+00:00 unitHistory [2018-01-27_01-00_Westport .csv]
2018-01-28 06:00:12+00:00 unitHistory [2018-01-28_01-00_Westport .csv]
2018-01-29 06:00:11+00:00 unitHistory [2018-01-29_01-00_Westport .csv]
2018-01-30 06:00:10+00:00 unitHistory [2018-01-30_01-00_Westport .csv]
2018-01-31 06:00:11+00:00 unitHistory [2018-01-31_01-00_Westport .csv]
2018-02-01 06:00:10+00:00 unitHistory [2018-02-01_01-00_Westport .csv]
2018-02-02 06:00:11+00:00 unitHistory [2018-02-02_01-00_Westport .csv]
2018-02-02 16:05:06+00:00 unitHistory [2018-02-02_11-04_Westport .csv]
2018-02-03 06:00:11+00:00 unitHistory [2018-02-03_01-00_Westport .csv]
2018-02-04 06:00:11+00:00 unitHistory [2018-02-04_01-00_Westport .csv]
2018-02-05 06:00:11+00:00 unitHistory [2018-02-05_01-00_Westport .csv]
2018-0

## Comments
Get the comments entered on WSV Application. This can be used to automate the intereaction with the application users. 

In [15]:
for comment in wsv.comments(unitGuid):
    print(f'{comment["author"]:<18} {comment["date"]} {comment["text"]}')

Stacey Hoppe       2018-08-17 20:27:43.124000+00:00 8&#x2F;17&#x2F;18- unit down inverter jesse will be at site 8&#x2F;18 to reset &#x28;sh&#x29;
Stacey Hoppe       2018-08-30 18:09:25.925000+00:00 8&#x2F;30&#x2F;2018- Unit down till 8&#x2F;31&#x2F;2018 FLS EngWtr out A, STP EngWtrOUtAhigh
Stacey Hoppe       2019-09-03 19:44:27.009000+00:00 unit down exhaust flex broken
Stacey Hoppe       2019-10-15 19:42:45.736000+00:00 Unit will be down till tomorrow need to investigate Cyl # head  &#x28;SH&#x29;
Stacey Hoppe       2019-12-02 19:01:01.698000+00:00 Unit will be down for 20,000 hour service
Stacey Hoppe       2020-09-08 22:01:36.014000+00:00 Inverter fault shut unit down
Stacey Hoppe       2020-09-23 17:26:23.423000+00:00 BOC Inverter Shutdown. can restart unit
Stacey Hoppe       2020-09-25 09:30:13.967000+00:00 Inverter fault


## Download
This can be used to download a file

In [16]:
FILENAME = 'filename.csv'

if wsv.download(unitGuid,FILENAME):
    print('SUCCESS')
else:
    print('FAILED')

API GET 'download' returned code: 404 (Not Found) 


FAILED


## Command
Controls the genset. The available commands are `start`,`stop`,`faultReset`,`changeMcb` (toggle mains circuit breaker), `changeGcb` (toggle genset circuit breaker) and `changeMode`.

For changeMode enter the mode parameter e.g. to `man` or `aut`

In [21]:
if wsv.command(unitGuid,'start'):
    print('Genset start succesful')
else:
    print('Genset start failed')

Genset start succesful


In [18]:
if wsv.command(unitGuid,'changeMode','aut'):
    print('Change mode succesful')
else:
    print('Change mode failed')

Change mode succesful


# Asynchronous package
The previous examples are using synchronous Python package `requests`. Whilst simple for understanding, the disadvantage of this package is, that it blocks computer resources while waiting for the server response.

There is therefore also Asynchronous module available. It has the same methods as described above, but they start with a word `_async`.  Explaining the `aiohttp` package is beyond the scope of this document, if you intend using the async package, I assume you are familiar with the concept.

Here is an example of usage - doing the same as `wsv.units()`

In [19]:
# import aiohttp
# import asyncio
# from comap import api_async

# async def main():
#     async with aiohttp.ClientSession() as session:
#         identity = api_async.Identity(session, secrets["COMAP_KEY"])
#         token = await identity.authenticate(
#             secrets["CLIENT_ID"], secrets["SECRET"]
#         )

#         if token is not None:
#             # Create WSV instance to call APIs
#             wsv = api_async.WSV(
#                 session,
#                 secrets["LOGIN_ID"],
#                 secrets["COMAP_KEY"],
#                 token["access_token"],
#             )
#             units = await wsv.units()
#             print("List of units available within user account")
#             print("-------------------------------------------")
#             for unit in units:
#                 print(f'{unit["unitGuid"]} : {unit["name"]}')

# asyncio.run(main())

*You can also refer to the [Jupyter asyncio tests](https://nbviewer.jupyter.org/github/bruxy70/ComAp-API/blob/development/Asyncio%20test.ipynb)*